In [1]:
# bike_station_id
# station_name
# city_code
# district
# disable
# near_to_bus
# near_to_mrt


In [2]:
import pandas as pd
import os
import geopy.distance
import numpy as np
from pathlib import Path

In [3]:
# bus_station = pd.read_csv('../bus/tpe_bus_station_info_after_api_call.csv')
mrt_station = pd.read_csv('../MRT/mrt_station.csv')
bike_station = pd.read_csv('../bike/bike_usage_realtime.csv',index_col=0)

In [4]:
bike_station['sarea'] = bike_station['sarea'].replace('臺大公館校區','大安區')

In [5]:
bike_station.rename(
    {
        'sno':'bike_station_id',
        'sna':'station_name',
        'act':'disable',
        'sarea':'district',
        'latitude':'lat',
        'longitude':'lng'

    }
    ,axis=1,
    inplace=True
                    )

In [6]:
bike_station['city_code']='TPE'

bike_station['station_name'] = bike_station['station_name'].apply(lambda x: x.split('_')[1])
bike_station = bike_station[['bike_station_id','station_name',
                             'city_code','district','disable','lng','lat']]

In [7]:
mrt_station.rename(
    {
        'StationPosition.PositionLon':'lng',
        'StationPosition.PositionLat':'lat'
    },
    axis=1,
    inplace=True
)

In [8]:
mrt_dist = []
# bus_dist = []
for idx, row in bike_station.iterrows():
    point_1 = row[['lat','lng']].to_numpy()
    mrt_dist.append(int(mrt_station[['lat','lng']].apply(lambda row:
                                 geopy.distance.geodesic(point_1,row.to_numpy()).m,
                                 axis=1).min()))
#     bus_dist.append(int(bus_station[['lat','lng']].apply(lambda row:
#                                  geopy.distance.geodesic(point_1,row.to_numpy()).m,
#                                  axis=1).min()))
# bike_station['near_to_bus'] = bus_dist
bike_station['near_to_mrt'] = mrt_dist
# del mrt_station,bus_station

In [9]:
bike_station.loc[:,'disable'] = ~bike_station['disable'].astype('bool')

In [12]:
ubike_rt = pd.read_csv('ubike_0501.csv',index_col=0)
ubike_rt.drop_duplicates(inplace=True)
ubike_rt['sarea'] = ubike_rt['sarea'].replace('臺大公館校區','大安區')
ubike_rt['sna'] = ubike_rt['sna'].apply(lambda x: x.split('_')[1])


In [13]:
ubike_rt['thres'] = np.min(np.column_stack(((ubike_rt['tot']*0.1).to_numpy(),np.full(len(ubike_rt),5))),axis=1)
def almost_empty(row):
    bike = row['sbi']
    thres = row['thres']
    # if bike <thres:
    if bike <2:
        return True
    else:
        return False

In [14]:
ubike_rt['almost_empty'] = ubike_rt.apply(almost_empty,axis=1)
ubike_rt['is_empty'] = ubike_rt['sbi']==0
ubike_rt['date'] = pd.to_datetime(ubike_rt['updateTime']).dt.date
ubike_rt['hour'] = pd.to_datetime(ubike_rt['updateTime']).dt.hour
ubike_rt['min'] = (pd.to_datetime(ubike_rt['updateTime']).dt.minute)//10*10

In [15]:
ubike_rt = ubike_rt.drop(['ar', 'aren', 'infoDate', 'infoTime', 'mday','updateTime',
       'srcUpdateTime','sarea', 'sareaen', 'snaen', 'sno'],axis=1)

In [21]:
ubike_rt = ubike_rt.merge(bike_station[['station_name','near_to_mrt']],
               left_on='sna',right_on='station_name',
               how='inner')

In [24]:
import datetime
def create_dt_from_row(row):
    year = row['date'].year
    month = row['date'].month
    day = row['date'].day
    return datetime.datetime(year,month,day,row['hour'],row['min'])

In [25]:
ubike_rt['timestamp'] = ubike_rt.apply(lambda row:create_dt_from_row(row),axis=1)

In [28]:
ubike_rt.to_csv('mrt_distance_empty_risk.csv',index=False)

In [30]:
ubike_rt

,bemp,lat,lng,sbi,sna,tot,thres,almost_empty,is_empty,date,hour,min,station_name,near_to_mrt,timestamp
0,23,25.02605,121.54360,5,捷運科技大樓站,28,2.8,False,False,2024-04-16,22,20,捷運科技大樓站,15,2024-04-16 22:20:00
1,24,25.02605,121.54360,4,捷運科技大樓站,28,2.8,False,False,2024-04-16,22,20,捷運科技大樓站,15,2024-04-16 22:20:00
2,23,25.02605,121.54360,5,捷運科技大樓站,28,2.8,False,False,2024-04-16,22,30,捷運科技大樓站,15,2024-04-16 22:30:00
3,23,25.02605,121.54360,5,捷運科技大樓站,28,2.8,False,False,2024-04-16,22,40,捷運科技大樓站,15,2024-04-16 22:40:00
4,5,25.02605,121.54360,23,捷運科技大樓站,28,2.8,False,False,2024-04-16,22,50,捷運科技大樓站,15,2024-04-16 22:50:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2668818,11,25.09601,121.53262,4,福林路100巷11弄口,15,1.5,False,False,2024-05-01,19,40,福林路100巷11弄口,702,2024-05-01 19:40:00
2668819,11,25.09601,121.53262,4,福林路100巷11弄口,15,1.5,False,False,2024-05-01,19,50,福林路100巷11弄口,702,2024-05-01 19:50:00
2668820,15,24.99442,121.53844,3,景福街223巷,18,1.8,False,False,2024-05-01,19,40,景福街223巷,288,2024-05-01 19:40:00
2668821,15,24.99442,121.53844,3,景福街223巷,18,1.8,False,False,2024-05-01,19,40,景福街223巷,288,2024-05-01 19:40:00
